In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Load the CSV file, without skipping any rows initially
file_path = "/Users/javanmardi/Work/IGSB/Bone2Gene_Survey_Study/3_progressive_results/B2G Survey_9_29_2024.csv"
initial_df = pd.read_csv(file_path, sep=';', header=None)

In [ ]:
# Set the first row as the header of the dataframe
headers = initial_df.iloc[0]
questions = initial_df.iloc[1]
header_question_dict = dict(zip(headers, questions))

In [ ]:
# Now, read the CSV again, this time skipping the first three rows and setting the first row as header
data_df = pd.read_csv(file_path, sep=';', skiprows=[1, 2])

# Show the first few rows of the dataframe to verify
# print("\nData Preview:")
data_df.head()

In [ ]:
#the finished surveys

finished_df=data_df[data_df['Finished']==True]
finished_df.shape

In [ ]:
# Identify empty columns
empty_columns = finished_df.isna().all()

# Drop empty columns
data = finished_df.drop(columns=empty_columns[empty_columns].index)

# Q13

In [ ]:
header_question_dict['Q13']

In [ ]:
# Calculate frequency of values in the column
Q13_value_counts = data['Q13'].value_counts().reset_index()
Q13_value_counts.columns = ['Value', 'Count']
Q13_value_counts

In [ ]:
# Mapping dictionary
country_mapping = {
    'US': 'United States',
    'United States of America': 'United States',
    'USA': 'United States',
    'usa': 'United States',
    'united states': 'United States',
    'germany': 'Germany',
    'Deutschland': 'Germany',
    'portugal': 'Portugal',
    'canada': 'Canada',
    'FR': 'France',
    'italy': 'Italy',
    'ITALY': 'Italy',
    'Moscow': 'Russia',
    'Slowenien': 'Slovenia',
    'Turkiye ':'Turkey',
    'TÃ¼rkiye':'Turkey',
    'BELGIUM':'Belgium',
    'Austria (Europe)':'Austria',
    'UK':'United Kingdom',
    'United Kingdom ':'United Kingdom'
}

# Replace the country names in the DataFrame
data['Q13'] = data['Q13'].replace(country_mapping)


# Figure

In [ ]:
# Create a world map using Plotly Express scatter_geo
fig = px.scatter_geo(Q13_value_counts, locations="Value", locationmode='country names', color="Count", size="Count",
                     projection="natural earth", color_continuous_scale="Jet_r", size_max=60)

# Center align the title
fig.update_layout(title={
    'text': 'The locations where the participants primarily work',
    'x': 0.5,  # Align center
    'xanchor': 'center',
    'yanchor': 'top',
    'font': {'size': 24}  # Increase font size
})

# Remove the margins
fig.update_layout(margin=dict(l=0, r=0, t=30, b=0))


fig.show()

fig.write_image("world_map.png", width=1200, height=700, scale=6)